
https://www.tensorflow.org/text/tutorials/text_generation

https://www.tensorflow.org/text/tutorials/nmt_with_attention

https://www.tensorflow.org/text/tutorials/transformer


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf


# Own transformer

In [2]:
!pip install tf-models-official

In [3]:
import tensorflow_models as tfm

In [4]:
t = tf.random.normal(shape=(1, 3, 4))
display(t)

t_layer = tfm.nlp.layers.Transformer(num_attention_heads=4, intermediate_size=4*4, intermediate_activation='relu')
display(t_layer(t))


<tf.Tensor: shape=(1, 3, 4), dtype=float32, numpy=
array([[[ 0.90928006,  0.05720628, -1.042153  ,  1.5268366 ],
        [ 1.2028786 , -1.7105056 , -0.8862882 , -0.32299075],
        [-0.1554523 , -1.1886663 ,  0.07023362, -1.7955782 ]]],
      dtype=float32)>

<tf.Tensor: shape=(1, 3, 4), dtype=float32, numpy=
array([[[ 0.75833327, -0.6728287 , -1.261066  ,  1.1755615 ],
        [ 1.5400293 , -1.257356  , -0.08045658, -0.20221666],
        [ 0.8475777 , -0.62565595,  1.0860231 , -1.3079449 ]]],
      dtype=float32)>

In [5]:
for sm in t_layer.submodules:
  if sm.name not in ['key', 'attention_output', 'query', 'value']:
    prefix = f'{t_layer.name}/{sm.name}/'
    print(sm.name)
    if 'weights' in dir(sm):
      for w in sm.weights:
        print(f'    {w.name[len(prefix):]}', '\t\t\t', w.shape)
    else:
      print(f'   ', sm.submodules)

dropout
self_attention
    query/kernel:0 			 (4, 4, 1)
    query/bias:0 			 (4, 1)
    key/kernel:0 			 (4, 4, 1)
    key/bias:0 			 (4, 1)
    value/kernel:0 			 (4, 4, 1)
    value/bias:0 			 (4, 1)
    attention_output/kernel:0 			 (4, 1, 4)
    attention_output/bias:0 			 (4,)
self_attention_layer_norm
    gamma:0 			 (4,)
    beta:0 			 (4,)
dropout_1
activation
intermediate
    kernel:0 			 (4, 16)
    bias:0 			 (16,)
output
    kernel:0 			 (16, 4)
    bias:0 			 (4,)
dropout_2
output_layer_norm
    gamma:0 			 (4,)
    beta:0 			 (4,)
dropout_3
softmax


# Донавчання - Transfer Learning

- Bert

- GPT

- T5

## Prompt Engineering

- Context
- Task
- Instruction
- Clarity
- Refine

# GPT 2

In [6]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, GPT2LMHeadModel

model_name = 'gpt2-large'  # gpt2 gpt2-medium gpt2-large

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
GPT2 = TFGPT2LMHeadModel.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id)

GPT2.summary()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  774030080 
 er)                                                             
                                                                 
Total params: 774030080 (2.88 GB)
Trainable params: 774030080 (2.88 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
input_sequense = "After coming to the work I"

prompt1 = """
  2+2=4
  16+5=21
  20+3=[endprompt]
  """

prompt2 = """
  Taras Shevchenko: Kaniv
  Ivan Franko: Lviv
  Lesea Ukrayinka:
  """

input_ids = tokenizer.encode(input_sequense, return_tensors='tf')

output = GPT2.generate(input_ids, max_length=30, num_beams=1, do_sample=True, temperature=1, num_return_sequences=1)


In [8]:
for out in tokenizer.batch_decode(output, skip_special_tokens=True):
  print(out)
  print('---------')

After coming to the work I was struck by how many people were affected by the fires that swept through my hometown of Madras, Chennai and neighboring regions
---------


In [9]:
from transformers import pipeline, set_seed
from torch import tensor
generator = pipeline('text-generation', model='gpt2', tokenizer=tokenizer)
set_seed(0)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


In [10]:
prompt_sentiment = """Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My day has been really great!
Sentiment: Positive
###
Text: Not a fan when it is cloudy
Sentiment:"""

prompt_addition = """Addition
Numbers: 2,2
Result: 4
###
Numbers: 16,5
Result: 21
###
Numbers: 41,1
Result:"""

prompt2 = """Burial city
###
Taras Shevchenko: Kaniv
###
Ivan Franko: Lviv
###
Lesea Ukrayinka:"""

# result = generator(prompt_sentiment, top_k=2, temperature=1, max_length=55)
# result = generator(prompt_addition, top_k=2, temperature=1, max_length=55)
result = generator(prompt2, top_k=2, temperature=1, max_length=55)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=55) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [11]:
print(result[0]['generated_text'])

Burial city
###
Taras Shevchenko: Kaniv
###
Ivan Franko: Lviv
###
Lesea Ukrayinka: Lviv
###
Ivan Franko: Lviv
###
Lesea Ukrayinka: Lviv
###
Lesea Ukrayinka: Lviv
###
Lesea Ukrayinka: Lviv
###
Lesea Ukrayinka: Lviv
###
Lesea Ukrayinka: Lviv
###
Lesea Ukrayinka: Lviv
###
Lesea Ukrayinka: Lviv
###
Lesea Ukrayinka: Lviv
###
Lesea Ukrayinka: Lviv
###
Lesea Ukrayinka: Lviv
###
Lesea Ukrayinka: Lviv
###
###
Lesea Ukrayinka: Lviv
###
###
###
###
Lesea Ukrayinka: Lviv
###
###
Lesea Ukrayinka: Lviv
###
###
###
Lesea Ukrayinka: Lviv
###
###
###
Lesea Ukrayinka: Lviv
###
###
###
###
Lesea Ukrayinka: Lviv
###
###
###
###
Lesea Ukrayinka:


In [12]:
to_summarize = """When I am dead, bury me
In my beloved Ukraine,
My tomb upon a grave mound high
Amid the spreading plain,
So that the fields, the boundless steppes,
The Dnieper's plunging shore
My eyes could see, my ears could hear
The mighty river roar."""

result = generator(
    f"""Summarization Task:\n{to_summarize}\nTL;DR:""",
    max_length=512,
    top_k=5,
    temperature=1,
    no_repeat_ngram_size=2
)

print(result[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summarization Task:
When I am dead, bury me
In my beloved Ukraine,
My tomb upon a grave mound high
Amid the spreading plain,
So that the fields, the boundless steppes,
The Dnieper's plunging shore
My eyes could see, my ears could hear
The mighty river roar.
TL;DR: I'm dead in my own home, in Ukraine. I was born and raised in this land, but I didn't know how to get to this place. This is a place of death.


<a href='./carier.png'>